<a href="https://colab.research.google.com/github/PabloRR100/intro_deep_learning/blob/main/hackathon/notebook/template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Similarity Engine

Build a similarity engine using pre-trained compute vision models

In [293]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [294]:
!ls "/content/drive/MyDrive/MasterPokemos"

db  embeddings.pickle  testing


In [295]:
# !unzip "/content/drive/MyDrive/Clases DL - UCM/Pablo/data.zip" -d /content/

In [296]:
# !ls "/content/data/database"Ç

In [297]:
import os
import torch
from PIL import Image as PILImage
from huggingface_hub import hf_hub_download, from_pretrained_keras
import tensorflow as tf
import numpy as np
import pickle
from pathlib import Path

https://huggingface.co/timm/mobilenetv3_small_100.lamb_in1k

In [298]:
import timm


In [299]:
# --- 1. Load Pretrained Model (Feature Extractor) ---
def get_model():
  return timm.create_model('mobilenetv3_small_100.lamb_in1k', pretrained=True)

In [300]:
model = get_model()

In [301]:
# --- 2. Compute Embedding for a Single Image ---
def get_embedding(model, img_path):
  img = PILImage.open(img_path)
  if img.mode =='RGBA':
    img = img.convert('RGB')
  elif img.mode =='L':
    img = img.convert('RGB')
  elif img.mode =='P':
    img = img.convert('RGB')
  # print(img.mode)

  model.eval()
  data_config = timm.data.resolve_model_data_config(model)
  transforms = timm.data.create_transform(**data_config, is_training=False)
  return model(transforms(img).unsqueeze(0))

In [302]:
# print(get_embedding(model,'/content/drive/MyDrive/MasterPokemos/bulbasaur/bulbasaur_1.jpeg' ))

In [303]:
def calcular_centroides(embeddings):
  for label, embeddings_list in embeddings.items():
    centroid = torch.zeros_like(embeddings_list[0]["embedding"])
    for item in embeddings_list:
      centroid += item["embedding"]
    centroid /= len(embeddings_list)
  return centroid

In [304]:
# --- 3. Walk Dataset Folder & Compute All Embeddings ---
def compute_embeddings(
    model: torch.nn.Module,
    data_root: str | os.PathLike,
):
    """
    Compute embeddings for all images in the dataset and store them in a dictionary
    where each Pokemon has a list of embeddings.

    Args:
        model: The feature extraction model
        data_root: Root directory containing Pokemon class folders

    Returns:
        Dictionary mapping Pokemon names to lists of embeddings

    Example:
    ```
    {
        "label": pokemon_name,
        "img_path": img_path,
        "embedding": emb,
    }
    ```
    """
    model.eval()
    results= {}
    for label in os.listdir(data_root):
      label_path = os.path.join(data_root, label)
      if os.path.isdir(label_path): # Check if it's a directory
        results[label] = []
        for img_name in os.listdir(label_path):
          img_path = os.path.join(label_path, img_name)
          if os.path.isfile(img_path): # Check if it's a file
            print(f"Processing: {img_path}")
            embedding = get_embedding(model, img_path)
            # results({"label": label, "img_path": img_path, "embedding": embedding})
            results[label].append({"label": label, "img_path": img_path, "embedding": embedding})
    return results

In [305]:
embeddings = compute_embeddings(model,'/content/drive/MyDrive/MasterPokemos/db')

Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/bulbasaur_1.jpeg
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/xzc.png
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/s.png
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/00000153.jpg
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/00000237.jpg
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/00000240.jpg
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/00000158.png
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/00000157.png
Processing: /content/drive/MyDrive/MasterPokemos/db/bulbasaur/00000155.gif
Processing: /content/drive/MyDrive/MasterPokemos/db/charmander/charmander.jpeg
Processing: /content/drive/MyDrive/MasterPokemos/db/charmander/00000001.jpg
Processing: /content/drive/MyDrive/MasterPokemos/db/charmander/0e2fe299821445639dc97b6fdef415be.jpg
Processing: /content/drive/MyDrive/MasterPokemos/db/charmander/0aaf393c1c21436

In [306]:
def export_embeddings_to_pickle(embeddings, path):
  with open(path, 'wb') as handle:
    pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [307]:
def load_embeddings_from_pickle(path):
  with open(path, 'rb') as handle:
    embeddings = pickle.load(handle)
  return embeddings

In [308]:
export_embeddings_to_pickle(embeddings, '/content/drive/MyDrive/MasterPokemos/embeddings.pickle')
# embeddings = load_embeddings_from_pickle('/content/drive/MyDrive/MasterPokemos/embeddings.pickle')
# embedding_test

In [309]:
def compute_similarity_for_image(img_path, expected_label, model, db):
    """
    Compute similarity scores between a test image and all database embeddings.
    Return

    Args:
        img_path: Path to the test image
        expected_label: Expected Pokemon label for the test image
        model: CNN/ViT model for feature extraction
        db: Database of Pokemon embeddings (loaded pickle)

    Example output:
    ```
    Expected: bulbasaur
    Top matches:
      bulbasaur    → similarity: 0.7646
      charmander   → similarity: 0.7179
      squirtle     → similarity: 0.7161
    ```
    """

In [336]:
# --- Single Image Similarity ---
def compute_similarity_for_image(img_path, expected_label, model, db):
    """
    Compute similarity scores between a test image and all database embeddings.
    Return

    Args:
        img_path: Path to the test image
        expected_label: Expected Pokemon label for the test image
        model: CNN/ViT model for feature extraction
        db: Database of Pokemon embeddings (loaded pickle)

    Example output:
    ```
    Expected: bulbasaur
    Top matches:
      bulbasaur    → similarity: 0.7646
      charmander   → similarity: 0.7179
      squirtle     → similarity: 0.7161
    ```
    """
    print(f"\n🔍 Testing image: {img_path}")
    test_emb = get_embedding(model, img_path)
    similarity_results = []
    for label, embeddings_list in db.items():
        for item in embeddings_list:
            db_emb = item["embedding"]
            sim_score = cosine_similarity(test_emb, db_emb)
            similarity_results.append({
                "label": item["label"],
                "img_path": item["img_path"],
                "similarity": sim_score
            })

    # por cada label, hacemos media de similarity
    # Calcular la similitud promedio por cada etiqueta
    average_similarities = {}
    label_counts = {}

    for result in similarity_results:
        label = result["label"]
        similarity = result["similarity"]

        if label not in average_similarities:
            average_similarities[label] = 0.0
            label_counts[label] = 0

        average_similarities[label] += similarity
        label_counts[label] += 1

    # Calcular el promedio final
    for label in average_similarities:
        if label_counts[label] > 0:
            average_similarities[label] /= label_counts[label]

    # Ordenar los promedios de similitud por etiqueta
    sorted_average_similarities = sorted(average_similarities.items(), key=lambda item: item[1], reverse=True)

    print(f"Expected: {expected_label}")
    print("Top matches:")

    for i, result in enumerate(sorted_average_similarities[:5]): # Displaying top 5 matches
        print(f"  {result[0]:<12s} → similarity: {result[1]:.4f}")


In [337]:
compute_similarity_for_image("/content/drive/MyDrive/MasterPokemos/testing/squirtle/squirtle_1.jpeg", "squirtle", model, embeddings)


🔍 Testing image: /content/drive/MyDrive/MasterPokemos/testing/squirtle/squirtle_1.jpeg
Expected: squirtle
Top matches:
  squirtle     → similarity: 0.9030
  bulbasaur    → similarity: 0.8811
  charmander   → similarity: 0.8699


In [338]:
# --- Dataset Loop ---
def compute_similarity_for_dataset(test_dir, db_path):
    """
    Compute similarity scores for all images in the test dataset.

    Args:
        test_dir: Directory containing test images organized by Pokemon
        db_path: Path to the pickle file containing database embeddings
    """
    db_embeddings = load_embeddings_from_pickle(db_path)
    # print(db_embeddings)
    processed_count = 0
    for root, _, files in os.walk(test_dir):
        expected_label = os.path.basename(root)
        if root != test_dir:
            for file in files:
                # Check if the file is an image (you might want a more robust check)

                img_path = os.path.join(root, file)
                similarity_results = compute_similarity_for_image(img_path, expected_label, model, db_embeddings)
                print(similarity_results)



In [339]:
compute_similarity_for_dataset(
    test_dir='/content/drive/MyDrive/MasterPokemos/testing/',
    db_path='/content/drive/MyDrive/MasterPokemos/embeddings.pickle',
)


🔍 Testing image: /content/drive/MyDrive/MasterPokemos/testing/bulbasaur/bulbasaur_1.jpeg
Expected: bulbasaur
Top matches:
  bulbasaur    → similarity: 0.8852
  charmander   → similarity: 0.8737
  squirtle     → similarity: 0.8565
None

🔍 Testing image: /content/drive/MyDrive/MasterPokemos/testing/squirtle/squirtle_1.jpeg
Expected: squirtle
Top matches:
  squirtle     → similarity: 0.9030
  bulbasaur    → similarity: 0.8811
  charmander   → similarity: 0.8699
None

🔍 Testing image: /content/drive/MyDrive/MasterPokemos/testing/charmander/charmander.jpeg
Expected: charmander
Top matches:
  charmander   → similarity: 0.9004
  squirtle     → similarity: 0.8811
  bulbasaur    → similarity: 0.8801
None


### Compute Metrics
- Add 3 more pokemon pictures for each pokemon in the testing folder.
- Compute the classification accuracy for each Pokemon